In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u
from lsdb.core.search import ConeSearch

import lsdb
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
cluster.adapt(minimum=1, maximum=16)
client = Client(cluster)
client

In [ ]:
ps1_path = "s3://stpubdata/panstarrs/ps1/public/hipscat/otmo"
ps1 = lsdb.read_hipscat(ps1_path, storage_options={'anon': True},
    columns=["objName","objID","rMeanApMag","raMean","decMean"])
#ps1.head()

In [ ]:
c = SkyCoord('19:22:40  +44:30:00', unit=(u.hourangle, u.deg))
cone_ra, cone_dec = c.ra.value, c.dec.value
radius_arcsec = 11500 # 340; 1060; 3350; 11500; 34000 arcsec
cone_filter = ConeSearch(cone_ra, cone_dec, radius_arcsec)

ztf_path = ("s3://irsa-mast-tike-spitzer-data/data/"
            "ZTF/dr20/objects/hipscat/ztf-dr20-objects-hipscat")
ztf_piece = lsdb.read_hipscat(ztf_path, search_filter=cone_filter)
#ztf_piece.compute()

In [ ]:
ztf_x_ps1 = ztf_piece.crossmatch(ps1, radius_arcsec=1, n_neighbors=1, suffixes=("_ztf", "_ps1"))

In [ ]:
%%time
xmatch = ztf_x_ps1.compute()

In [ ]:
xmatch